In [1]:
!pip install -Uq openai-agents

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.9/121.9 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.3/129.3 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.1/131.1 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 1.0 MB/s eta 0:00:00


In [2]:
import nest_asyncio
nest_asyncio.apply()

In [3]:
from pydantic import BaseModel
from agents import (
    AsyncOpenAI,
    OpenAIChatCompletionsModel,
    RunConfig
)

In [4]:
from google.colab import userdata
gemini_api_key = userdata.get("GEMINI_API_KEY")

external_client = AsyncOpenAI(
    api_key=gemini_api_key,
    base_url="https://generativelanguage.googleapis.com/v1beta/openai/",
)

model = OpenAIChatCompletionsModel(
    model="gemini-2.0-flash",
    openai_client=external_client
)

config = RunConfig(
    model=model,
    model_provider=external_client,
    tracing_disabled=True
)

In [5]:
import asyncio
from agents import Agent, Runner

In [8]:
support_billing_agent = Agent(
    name="Billing Support Agent",
    instructions="You handle billing-related queries such as payments, invoices, and account charges."
)

support_technical_agent = Agent(
    name="Technical Support Agent",
    instructions="You handle technical issues like app crashes, login problems, and system bugs."
)

triage_agent = Agent(
    name="Support Triage Agent",
    instructions=(
        "Determine whether the customer's issue is technical or billing-related. "
        "If it's about payments, subscriptions, or charges, hand off to Billing Support Agent. "
        "If it's about bugs, errors, or system access, hand off to Technical Support Agent."
    ),
    handoffs=[support_billing_agent, support_technical_agent],
)


async def main(input: str):
    result = await Runner.run(triage_agent, input=input, run_config=config)
    print(result.final_output)

In [9]:
asyncio.run(main("I was charged twice for my monthly subscription."))

I'm sorry to hear you were charged twice. I can definitely help you look into that. To investigate, I'll need a little more information. Could you please provide me with:

*   **The email address associated with your account:** This will help me quickly locate your subscription.
*   **The dates of both charges:** Knowing when the charges occurred will help me pinpoint them in our system.
*   **The amounts of both charges:** This will help confirm we're looking at the correct transactions.
*   **The last four digits of the card used for the payment:** This will help verify the payment method.

Once I have this information, I'll be able to investigate the charges and get this resolved for you as quickly as possible.

